**The target of this notebook is to quickly gain insights of a dataset and build a competitive model in a stipulated time to check basic knowledge
I hope you will like it.This gives a competitive score and actually a pretty good one in first try within an hour.
This notebook is useful for people who quickly want to know how to approach a data science problem.Do upvote if u like it.**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier

In [ ]:
pd.pandas.set_option('display.max_columns',None)
train=pd.read_csv("/kaggle/input/titanic/train.csv")
test=pd.read_csv("/kaggle/input/titanic/test.csv")

In [ ]:
train.head()

In [ ]:
train.shape

We have 10 features( exclusing passenger id and survived)

In [ ]:
test.shape

In [ ]:
train.info()

In [ ]:
#number of passengers survived
sum(list(train.Survived))

342 survived out of 891

**DATA EXPLORATION**

In [ ]:
train.describe()

Only 38% people survived

Lets try to understand how survival varies with different features

In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Higher class survived more

In [ ]:
train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

For anyone who has seen the movie this should not come as a surprise.(obviously there was space on wood plank for Leonardo DiCaprio)

In [ ]:
train[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

Believe me this is the most interesting thing i found.Dont want to draw any conclusion.But do think wisely.

In [ ]:
train[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
g = sns.FacetGrid(train, col='Survived')
g.map(plt.hist, 'Age', bins=20)

Children were saved more

I know what you want next.Here we go

In [ ]:
plt.figure(figsize=(14, 6))
plt.subplot(1,2,1)
cols = ['blue', 'lightcoral']
train['Sex'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',shadow=True, colors=cols)
plt.title('Total Male/Female onboard')
plt.subplot(1,2,2)
sns.barplot(x="Sex", y="Survived", data=train,palette='plasma')
plt.title('Sex vs Survived')
plt.ylabel("Survival Rate")
plt.show()


Lets move ahead

But do note-gender seems an important feature for prediction

In [ ]:
train.groupby(["Survived"]).Fare.mean()


higher fare people survived more

**Feature selection**

In [ ]:
#Correlation:  Its the most basic way to find relation between any two quantities.
corr = train.corr()

plt.figure(figsize=(15,15))

sns.heatmap(corr,vmax=0.9,square=True)
plt.show()

In [ ]:
train.corr()["Survived"]

Fare,number of siblings/spouses,parents/children,age,Pcclass seems important.
(For newbies: Negative  sign in correlation just means that quantities are inversely proprtional, they can be negative and important)
passenger ID is obviously irrelevant

**Handling missing values**

In [ ]:
train.isnull().mean()

In [ ]:
#lets drop cabin
train.drop("Cabin", axis = 1, inplace = True)
test.drop("Cabin", axis = 1, inplace = True)

In [ ]:
#age can be filled by median
train["Age"].fillna(train["Age"].median(), inplace = True)
test["Age"].fillna(test["Age"].median(), inplace = True) 

In [ ]:
#embarked can be filled by median/or mode
train['Embarked'].value_counts(normalize=True)

In [ ]:
Embarked_mode=train['Embarked'].mode()[0]
Embarked_mode

In [ ]:

train["Embarked"].fillna("S", inplace = True)

In [ ]:
test.isnull().mean()

In [ ]:
#fare can be filled with median
test["Fare"].fillna(test["Fare"].median(), inplace = True)

In [ ]:
test.isnull().mean()

In [ ]:
train.isnull().mean()

**Converting categorical to numeric**

In [ ]:
train['Sex'] = train['Sex'].map({'female': 0, 'male': 1})
test['Sex']= test['Sex'].map({'female': 0, 'male': 1})

train['Embarked'] = train['Embarked'].map({'S': 0, 'C': 1,'Q': 2})
test['Embarked']= test['Embarked'].map({'S': 0, 'C': 1,'Q': 2})

**Lets drop and combine some features now**

In [ ]:
train.drop(["Name","Ticket"], axis = 1, inplace = True)
test.drop(["Name","Ticket"], axis = 1, inplace = True)

Sib/Sp and Parch can be combined

In [ ]:
train["Family"] = train["SibSp"] + train["Parch"] + 1
test["Family"] = test["SibSp"] + test["Parch"] + 1
train=train.drop(["SibSp","Parch"],axis=1)
test=test.drop(["SibSp","Parch"],axis=1)
print(train.shape)
print(test.shape)

Feature Scaling

In [ ]:
scaler = StandardScaler()

train[['Age','Fare']] = scaler.fit_transform(train[['Age','Fare']])
test[['Age','Fare']] = scaler.transform(test[['Age','Fare']])

train.head()

In [ ]:
test.head()


**MODELS**

In [ ]:
X_train = train.drop(['Survived','PassengerId'], axis=1)
y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1)
X_train.shape, y_train.shape, X_test.shape

1.)Basic logistic regression

In [ ]:
# Logistic Regression

LR = LogisticRegression()
LR.fit(X_train, y_train)

# Making Predictions
y_pred = LR.predict(X_test)

In [ ]:
# Calculating the Accuracy of the model.

print("Accuracy:",round(LR.score(X_train, y_train)*100,2))

2.)Random forest classifier

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
Y_pred = random_forest.predict(X_test)

In [ ]:
#Checking accuracy
random_forest.score(X_train, y_train)

acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
acc_random_forest

In [ ]:
Features=['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Family']
feature_importance = pd.Series(random_forest.feature_importances_,index=Features).sort_values(ascending=False)
feature_importance

Well family doesnt seems to be that important.(Only as a feature here)(Highly pun intended.)

Lets try to make a new feature

In [ ]:
combine = [train, test]

In [ ]:


train[['Family', 'Survived']].groupby(['Family'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['Family'] == 1, 'IsAlone'] = 1

train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

This seems better than Family feature

In [ ]:
X_train = train.drop(['Survived','PassengerId'], axis=1)
y_train = train["Survived"]
X_test  = test.drop("PassengerId", axis=1)
X_train.shape, y_train.shape, X_test.shape

In [ ]:
random_forest_2 = RandomForestClassifier(n_estimators=100)
random_forest_2.fit(X_train, y_train)
Y_pred = random_forest_2.predict(X_test)

In [ ]:
#Checking accuracy
random_forest_2.score(X_train, y_train)

acc_random_forest_2 = round(random_forest_2.score(X_train, y_train) * 100, 2)
acc_random_forest_2

In [ ]:
Features=['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Family',"IsAlone"]
feature_importance = pd.Series(random_forest_2.feature_importances_,index=Features).sort_values(ascending=False)
feature_importance

Well Is Alone doesnt seems as important as we thought.
Neither removing Embarked helped

In [ ]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': Y_pred})

output.to_csv('my_submission_2.csv', index=False)
print("Your submission was successfully saved!")

**3.)XGBoost**

In [ ]:
# Instantiate our model
xg = XGBClassifier()
xg.fit(X_train, y_train)
xg_predictions = xg.predict(X_test)
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': xg_predictions})

output.to_csv('my_submission_3.csv', index=False)
print("Your submission was successfully saved!")


In [ ]:
xg.score(X_train,y_train)

This is also poorer than random forest classifier.
Without using other models i can confidently say that Random forest classifier is indeed the best.
Feature engineering could be better.
"Data is fuel to the model, Make sure it is good quality"
Well i clearly did this notebook in a stipulated time limit,thus this was the best i can offer.
Suggestions to improve are welcome.
Happy learning.